In [8]:
# This program retrieves ["ID", "Uniprot_code", "Gene_name", "Pfam_code", "Transmembrane_Regions"] of the
# original Uniprot file and gives a list of list which contain all this information

# This list of list with Panda's help will be converted into a DataFrame and then exported as .txt file

In [6]:
# Import necessary libraries
import pandas as pd

In [31]:
# Creates 2 lists 1 list of lists in wich every wanted item will be added and another
# which will be used as a template to append found items in its correct position
l_all_data = [["Gene(s)", "Lenght(AA)", "Disease"]] 
l_ind_data = ["", "", ""] 

# Open and read the original uniprot file retrieved from uniprot
f_in = open("uniprot-organism__Homo+sapiens+(Human)+[9606]_+reviewed_yes+anno--.txt", "r")
f_data = f_in.readlines ()

# This for retrieves the information wanted accessing by startswith and using filter to clear white spaces
l_disease = []
switch = 0 # desactivat

for data in f_data:
    
    if data.startswith("ID"):
        #print(data)
        mod_data = data.split(" ")
        mod_data = list(filter (None, mod_data)) # filter deletes all white spaces in a list
        mod_data2 = mod_data[3]
        l_ind_data[1] = mod_data2
        
    if data.startswith("GN   N"):
        mod_data = data.split (" ") 
        mod_data = list (filter(None, mod_data))
        mod_data = mod_data[1].replace('Name=','')
        mod_data = mod_data.replace(';', '')
        mod_data = mod_data.replace('\n', '')
        l_ind_data[0] = mod_data   
        
# Adding the  gene ID  and the disease
    if data.startswith("CC   -!- DISEASE:"):
        switch = 1 # activat
    
    if data.startswith("CC   -!- SIMILARITY:"):
        #print(l_disease)
        sw = 0
        j= ''
        l_omim = []
        for dis in l_disease:
            dis = dis.replace('\n', '').replace('CC       ','')
            dis = dis.strip()
            #print(dis)
            if 'CC   -!- DISEASE:' in dis:
                j = dis
                sw = 1
            if 'CC   -!- DISEASE:' in dis and '[MIM:' in dis:
                if '[MIM:' in j and 'CC   -!- DISEASE:'  in j and j not in l_omim:
                    l_omim.append(j)
                sw = 0
                j= ''
            if '[MIM:' in dis and 'CC   -!- DISEASE:' not in dis:
                j = j + ' ' + dis
                if '[MIM:' in j and 'CC   -!- DISEASE:'  in j and j not in l_omim:
                    l_omim.append(j)
                sw = 0
                j= ''
            if sw == 1 and dis not in j:
                j = j + ' ' + dis
        #print(l_omim)
        for i, omi in enumerate(l_omim):
            omi = omi.replace('CC   -!- DISEASE:','').strip()
            l_aux = omi.split(']:')
            omi = l_aux[0] + ']'
            l_omim[i] = omi
        
        #print(l_omim)
        l_ind_data[2] = ';'.join(l_omim)
        l_disease = []
        switch = 0 # desactivat

    if data.startswith("//"): #Netejem info de 1 ind data
        #print (l_ind_data) 
        l_all_data.append(l_ind_data)
        l_ind_data = ["", "", ""] 
        
    if switch == 1:   
        l_disease.append(data)
    
    


# Then pandas is used to get the information in a .txt file with 'myOutFile(Uniprot).txt' name
df=pd.DataFrame.from_records(l_all_data)
#print(df)
df.to_csv('Uniprot_lenght(AA)+disease.txt', header=False, index=None, sep=' ', mode='w')
